In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.color_palette("Set2")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data_columns = data.columns.tolist()
data_columns

In [ ]:
data.nunique()

In [ ]:
plt.figure(figsize=(15,10))
corr = data.corr()
sns.heatmap(corr,annot=True)

In [ ]:
catgory_columns = [cat for cat in data_columns if len(data[cat].unique()) < 6]
num_columns = [num for num in data_columns if len(data[num].unique()) > 6]
print(catgory_columns)
print(num_columns)

In [ ]:
plt.figure(figsize=(16,10))
for i in range(len(catgory_columns[:-1])):
    plt.subplot(2,4,i+1)
    sns.countplot(data=data,x=data[catgory_columns[i]],hue='output')

In [ ]:
sns.countplot(x=data['output'],data=data)

In [ ]:
plt.figure(figsize=(16,10))
for i in range(len(num_columns)):
    plt.subplot(2,3,i+1)
    sns.kdeplot(data=data,x=data[num_columns[i]],hue='output')

In [ ]:
plt.figure(figsize=(16,10))
for i in range(len(num_columns)):
    plt.subplot(2,3,i+1)
    sns.boxplot(data=data,x=data['sex'],y=data[num_columns[i]])

In [ ]:
data = data.drop(data[data['trtbps'] > 180].index)
data = data.drop(data[data['chol'] > 500].index)
print(len(data[data['trtbps'] > 180]))
print(len(data[data['chol'] > 500]))

In [ ]:
plt.figure(figsize=(18,15))
for i in range(len(num_columns)):
    plt.subplot(2,3,i+1)
    sns.lineplot(y=data['output'],x=data[num_columns[i]],hue=data['sex'])

In [ ]:

pre_data = data.copy()
data = pd.get_dummies(data=data,columns=catgory_columns[:-1])
data.head()

In [ ]:
pre_data

# Divide the data into two parts
# one is one-hot encoding
# the other is one-hot encoding not
# check the effect of the model

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
fit_data_num_columns = scaler.fit(data[num_columns])
fit_pre_data_num_columns = scaler.fit(pre_data[num_columns])
data[num_columns] = fit_num_columns.transform(data[num_columns])
pre_data[num_columns] = fit_pre_data_num_columns.transform(pre_data[num_columns])
data.head()

In [ ]:
pre_data.head()

In [ ]:
pre_x = pre_data.drop(['output'],axis=1)
pre_y = pre_data['output']
x = data.drop(['output'],axis=1)
y = data['output']
pre_x

In [ ]:
x

#     LR

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [ ]:
np.random.seed(10)
pre_LR = LogisticRegression()
pre_LR_cross = cross_val_score(pre_LR,pre_x,pre_y, cv = 3,scoring = "recall")
print(pre_LR_cross)
print('recall_mean:')
print(np.mean(pre_LR_cross)*100)
pre_LR_cross = np.mean(pre_LR_cross)*100

In [ ]:
LR = LogisticRegression()
LR_cross = cross_val_score(LR,x,y, cv = 3,scoring = "recall")
print(LR_cross)
print('recall_mean:')
print(np.mean(LR_cross) * 100)
LR_cross = np.mean(LR_cross) * 100

# SVM

In [ ]:
from sklearn.svm import SVC
pre_svc = SVC()
pre_svc_cross = cross_val_score(pre_svc,pre_x,pre_y, cv = 3,scoring = "recall")
print(pre_svc_cross)
print('recall_mean:')
print(np.mean(pre_svc_cross)*100)
pre_svc_cross = np.mean(pre_svc_cross)*100

In [ ]:
svc = SVC()
svc_cross = cross_val_score(svc,x,y, cv = 3,scoring = "recall")
print(svc_cross)
print('recall_mean:')
print(np.mean(svc_cross)*100)
svc_cross = np.mean(svc_cross)*100

# XGboost

In [ ]:
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")
pre_xgb = XGBClassifier()
pre_xgb_cross = cross_val_score(pre_xgb,pre_x,pre_y, cv = 3,scoring = "recall")
print(pre_xgb_cross)
print('recall_mean:')
print(np.mean(pre_xgb_cross)*100)
pre_xgb_cross = np.mean(pre_xgb_cross)*100

In [ ]:
xgb = XGBClassifier()
xgb_cross = cross_val_score(xgb,x,y, cv = 3,scoring = "recall")
print(xgb_cross)
print('recall_mean:')
print(np.mean(xgb_cross)*100)
xgb_cross = np.mean(xgb_cross)*100


In [ ]:
model_data_reall = {"data_no_one_hot_recall":[pre_LR_cross,pre_svc_cross,pre_xgb_cross],"data_one_hot_recall":[LR_cross,svc_cross,xgb_cross]}
model_score = pd.DataFrame(model_data_reall, index=['LR','SVC','XGB'])
model_score

1. The highest score for the model is svc ：recall 92.6%
2. The bottom score of the model is svc： recall 84.1%
3. There is not much difference in LR model scores
4. I hope you all experts can advise me a lot